In [1]:
!pip install praat-parselmouth
# 필요한 라이브러리 설치
!pip install pydub

In [2]:
from pydub import AudioSegment

# M4A 파일을 WAV로 변환
audio = AudioSegment.from_file("real_test.m4a", format="m4a")
audio.export("real_test.wav", format="wav")


<_io.BufferedRandom name='real_test.wav'>

음성 분석

인간 음성 분석

In [3]:
#인간의 음역대만 따와서 필터링한다. 여유시간을 둬서 완성도 있게 필터링함.
import parselmouth
import scipy.io.wavfile as wav
import numpy as np

def process_and_save_filtered_audio(input_file_path, output_file_path, human_voice_range=(30, 255), extend_ms=50, silence_after_ms=150):
    """
    입력 WAV 파일에서 인간 음역대만 필터링하여 결과를 반환하고, 새 파일로 저장하는 함수.

    Args:
        input_file_path (str): 입력 WAV 파일 경로.
        output_file_path (str): 출력 필터링된 WAV 파일 경로.
        human_voice_range (tuple): 인간 음역대 (최소 Hz, 최대 Hz).
        extend_ms (int): 유지할 길이 (밀리초).
        silence_after_ms (int): 여유 시간 (밀리초).

    Returns:
        filtered_data (numpy.ndarray): 필터링된 오디오 데이터.
        pitch_values (numpy.ndarray): 피치 값 배열.
        time_steps (numpy.ndarray): 시간 단계 배열.
    """
    # 1. WAV 파일 로드
    sampling_rate, data = wav.read(input_file_path)

    # 스테레오일 경우 단일 채널로 변환
    if len(data.shape) > 1:
        data = data.mean(axis=1).astype(np.int16)

    # 2. parselmouth로 음성 분석
    snd = parselmouth.Sound(data, sampling_rate)

    # 피치 분석
    pitch = snd.to_pitch()

    # 피치 값과 해당 시간 단계 가져오기
    pitch_values = pitch.selected_array["frequency"]
    time_steps = pitch.xs()

    # 3. 히스테리시스 필터 적용
    filtered_data = np.zeros_like(data, dtype=np.float32)
    prev_in_range = False

    # 샘플 수로 변환
    extend_samples = int(sampling_rate * extend_ms / 1000)
    silence_samples = int(sampling_rate * silence_after_ms / 1000)

    for i, t in enumerate(time_steps):
        index = int(t * sampling_rate)

        # 현재 피치가 인간 음역대에 해당하는지 확인
        in_range = human_voice_range[0] <= pitch_values[i] <= human_voice_range[1]

        if in_range or prev_in_range:
            # 앞뒤로 extend_ms만큼 구간을 유지
            start = max(0, index - extend_samples)
            end = min(len(data), index + extend_samples + silence_samples)
            filtered_data[start:end] = data[start:end]

        prev_in_range = in_range

    # 4. 결과 WAV 파일 저장
    wav.write(output_file_path, sampling_rate, filtered_data.astype(np.int16))

    print(f"{output_file_path} 파일이 생성되었습니다.")
    return sampling_rate,filtered_data, pitch_values, time_steps




In [4]:
# 예시 사용
sampling_rate, filtered_data, pitch_values, time_steps = process_and_save_filtered_audio(
    input_file_path="real_test.wav",
    output_file_path="real_test_only.wav"
)

real_test_only.wav 파일이 생성되었습니다.


tts 음성 분석

In [5]:
import parselmouth
import scipy.io.wavfile as wav
import numpy as np

def extract_pitch_from_tts(file_path):
    """
    TTS 음성을 분석하여 피치 값을 추출하는 함수.

    매개변수:
    file_path (str): TTS 음성 파일 경로 (WAV 형식).

    반환값:
    tuple: (샘플링 레이트, 피치 값 배열, 시간 단계 배열)
    """
    # 1. WAV 파일 로드
    tts_sampling_rate, tts_data = wav.read(file_path)

    # 스테레오일 경우 단일 채널로 변환
    if len(tts_data.shape) > 1:
        tts_data = tts_data.mean(axis=1).astype(np.int16)

    # 2. parselmouth로 음성 분석
    snd = parselmouth.Sound(tts_data, tts_sampling_rate)

    # 피치 분석
    pitch = snd.to_pitch()

    # 피치 값과 해당 시간 단계 가져오기
    pitch_values_tts = pitch.selected_array["frequency"]
    time_steps_tts = pitch.xs()

    # 피치 값과 시간 단계 배열 반환
    return tts_sampling_rate, tts_data, pitch_values_tts, time_steps_tts




In [6]:
# 예시 사용
file_path = "seoul_test4.wav"  # 분석할 TTS 음성 파일 경로
tts_sampling_rate, tts_data, pitch_values_tts, time_steps_tts = extract_pitch_from_tts(file_path)

타임스탬프

In [7]:
!pip install git+https://github.com/m-bain/whisperx.git
!pip install ffmpeg
!pip install torch

  Cloning https://github.com/m-bain/whisperx.git to /tmp/pip-req-build-delmhwqu
  Running command git clone --filter=blob:none --quiet https://github.com/m-bain/whisperx.git /tmp/pip-req-build-delmhwqu
  Resolved https://github.com/m-bain/whisperx.git to commit 9e3a9e0e38fcec1304e1784381059a0e2c670be5
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
  Using cached transformers-4.45.2-py3-none-any.whl.metadata (44 kB)
  Using cached transformers-4.45.1-py3-none-any.whl.metadata (44 kB)
  Using cached transformers-4.45.0-py3-none-any.whl.metadata (44 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.4 MB/s eta 0:00:00
  Using cached transformers-4.44.1-py3-none-any.whl.metadata (43 kB)
  Using cached transformers-4.44.0-py3-none-any.whl.metadata (43 kB)
  Using cached transformers-4.43.4-py3-none-any.whl.metadata (43 kB)
INFO: pip is 

타임스탬프를 구하는 함수

In [10]:
import whisperx
import gc
import torch

torch.backends.cudnn.benchmark = True

def extract_word_timestamps(audio_file, model_size="large-v2", device='cuda', batch_size=16, compute_type="float16"):

    #print("Is CUDA available:", torch.cuda.is_available())
    #print("Number of CUDA devices:", torch.cuda.device_count())

    # 1. Whisper 모델 로드 및 음성 텍스트 변환
    model = whisperx.load_model(model_size, device, compute_type=compute_type)

    # 오디오 파일 로드
    audio = whisperx.load_audio(audio_file)
    result = model.transcribe(audio, batch_size=batch_size, language="ko")

    # 2. Whisper 출력 정렬
    model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
    result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)

    # 3. 단어별 타임스탬프 리스트 생성
    timestamps = []
    for segment in result["segments"]:
        for word in segment["words"]:
            timestamps.append({
                "word": word["word"],
                "start": word["start"],
                "end": word["end"]
            })

    return timestamps



사용자 타임스탬프

In [11]:

# 예시 사용
audio_file_path = "real_test_only.wav"
word_intervals = extract_word_timestamps(audio_file_path, model_size="large-v2")

# 결과 출력
for item in word_intervals:
    print(f"단어: {item['word']}, 시작 시간: {item['start']}, 끝 시간: {item['end']}")

No language specified, language will be first be detected for each audio file (increases inference time).


100%|█████████████████████████████████████| 16.9M/16.9M [00:01<00:00, 15.4MiB/s]
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.5.0+cu121. Bad things might happen unless you revert torch to 1.x.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/2.31k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/18.2k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of the model checkpoint at kresnik/wav2vec2-large-xlsr-korean were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at kresnik/wav2vec2-large-xlsr-korean and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRA

단어: 안녕하세요., 시작 시간: 0.401, 끝 시간: 1.063
단어: 저는, 시작 시간: 1.304, 끝 시간: 1.464
단어: 지금, 시작 시간: 1.565, 끝 시간: 1.765
단어: 서울말을, 시작 시간: 1.865, 끝 시간: 2.287
단어: 구사하는, 시작 시간: 2.387, 끝 시간: 2.828
단어: 중입니다., 시작 시간: 2.949, 끝 시간: 3.43
단어: 이거, 시작 시간: 4.032, 끝 시간: 4.213
단어: 어디까지, 시작 시간: 4.253, 끝 시간: 4.654
단어: 올라가는, 시작 시간: 4.714, 끝 시간: 5.115
단어: 거예요?, 시작 시간: 5.196, 끝 시간: 5.998


tts 타임스탬프

In [13]:

# 예시 사용
audio_file_path = "seoul_test4.wav"
tts_word_intervals = extract_word_timestamps(audio_file_path, model_size="large-v2")

# 결과 출력
for item in tts_word_intervals:
    print(f"단어: {item['word']}, 시작 시간: {item['start']}, 끝 시간: {item['end']}")

No language specified, language will be first be detected for each audio file (increases inference time).


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.5.0+cu121. Bad things might happen unless you revert torch to 1.x.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at kresnik/wav2vec2-large-xlsr-korean were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC 

단어: 안녕하세요., 시작 시간: 0.009, 끝 시간: 0.732
단어: 저는, 시작 시간: 1.054, 끝 시간: 1.335
단어: 지금, 시작 시간: 1.476, 끝 시간: 1.737
단어: 서울말을, 시작 시간: 1.897, 끝 시간: 2.36
단어: 구사하는, 시작 시간: 2.48, 끝 시간: 2.862
단어: 중입니다., 시작 시간: 2.942, 끝 시간: 3.384
단어: 이거, 시작 시간: 4.027, 끝 시간: 4.288
단어: 어디까지, 시작 시간: 4.369, 끝 시간: 4.911
단어: 올라가는, 시작 시간: 4.971, 끝 시간: 5.373
단어: 거예요?, 시작 시간: 5.453, 끝 시간: 6.237


음성 비교 하기

1. 진폭차이로 구하기

In [16]:
def compare_amplitude_differences(user_word_intervals, tts_word_intervals, filtered_data, tts_data, threshold):
    # 데이터 타입을 float으로 변환하여 오버플로우 방지
    filtered_data = filtered_data.astype(np.float32)
    tts_data = tts_data.astype(np.float32)

    for user_word, tts_word in zip(user_word_intervals, tts_word_intervals):
        # 사용자 음성 단어 세그먼트의 시작과 끝 인덱스 계산
        user_start_idx = int(user_word['start'] * sampling_rate)
        user_end_idx = int(user_word['end'] * sampling_rate)

        # TTS 음성 단어 세그먼트의 시작과 끝 인덱스 계산
        tts_start_idx = int(tts_word['start'] * tts_sampling_rate)
        tts_end_idx = int(tts_word['end'] * tts_sampling_rate)

        # 사용자 음성의 진폭 값 중 100 이상인 값만 필터링
        user_amplitude = filtered_data[user_start_idx:user_end_idx]
        user_amplitude = user_amplitude[user_amplitude >= 100]

        # TTS 음성의 진폭 값 중 100 이상인 값만 필터링
        tts_amplitude = tts_data[tts_start_idx:tts_end_idx]
        tts_amplitude = tts_amplitude[tts_amplitude >= 100]

        # 사용자와 TTS 음성의 최대 및 최소 진폭 계산 (필터링된 값만 사용)
        if len(user_amplitude) > 0 and len(tts_amplitude) > 0:
            user_max_amplitude = np.max(user_amplitude)
            user_min_amplitude = np.min(user_amplitude)
            user_amplitude_difference = user_max_amplitude - user_min_amplitude

            tts_max_amplitude = np.max(tts_amplitude)
            tts_min_amplitude = np.min(tts_amplitude)
            tts_amplitude_difference = tts_max_amplitude - tts_min_amplitude

            # 진폭 차이 비교
            amplitude_diff = abs(user_amplitude_difference - tts_amplitude_difference)
            if amplitude_diff > threshold:
                print(f"단어: {user_word['word']}, 사용자 진폭 차이: {user_amplitude_difference}, TTS 진폭 차이: {tts_amplitude_difference}, 차이: {amplitude_diff}")


In [18]:
# 예제 사용
threshold_value = 8000  # 임계값 설정
compare_amplitude_differences(word_intervals, tts_word_intervals, filtered_data, tts_data, threshold_value)

단어: 저는, 사용자 진폭 차이: 5246.0, TTS 진폭 차이: 30366.0, 차이: 25120.0
단어: 지금, 사용자 진폭 차이: 7729.0, TTS 진폭 차이: 18225.0, 차이: 10496.0
단어: 올라가는, 사용자 진폭 차이: 14179.0, TTS 진폭 차이: 23831.0, 차이: 9652.0


2. 피치 차이로 구하기

In [19]:
import numpy as np

def calculate_pitch_differences(word_intervals, tts_word_intervals, pitch_values, time_steps, pitch_values_tts, time_steps_tts, threshold=25):
    """
    사용자와 TTS 단어의 피치 차이를 계산하고 결과를 반환하는 함수.

    Args:
        word_intervals (list): 사용자의 단어 간격 리스트.
        tts_word_intervals (list): TTS 단어 간격 리스트.
        pitch_values (np.array): 사용자 음성의 피치 값 배열.
        time_steps (np.array): 사용자 음성의 타임스탬프 배열.
        pitch_values_tts (np.array): TTS 음성의 피치 값 배열.
        time_steps_tts (np.array): TTS 음성의 타임스탬프 배열.
        threshold (int): 피치 차이 임계값.

    Returns:
        list: 피치 차이가 임계값을 초과하는 단어의 결과 리스트.
    """
    results = []

    # 사용자 단어와 TTS 단어의 피치 차이 계산
    for i in range(min(len(word_intervals), len(tts_word_intervals))):  # 단어 개수의 최소값으로 루프
        user_word = word_intervals[i]  # 사용자의 단어
        tts_word = tts_word_intervals[i]  # TTS 단어

        # 사용자 피치 샘플링 추출 (0이 아닌 값만 선택)
        user_pitch_samples = pitch_values[(time_steps >= user_word['start']) & (time_steps <= user_word['end'])]
        user_time_samples = time_steps[(time_steps >= user_word['start']) & (time_steps <= user_word['end'])]
        user_pitch_samples = user_pitch_samples[user_pitch_samples > 0]  # 0이 아닌 값만 포함

        if len(user_pitch_samples) > 0:
            user_pitch_max = np.max(user_pitch_samples)
            user_pitch_min = np.min(user_pitch_samples)
            user_pitch_diff = user_pitch_max - user_pitch_min
            user_max_time = user_time_samples[np.argmax(user_pitch_samples)]  # 최대 피치에 해당하는 시간

            # TTS 피치 샘플링 추출 (0이 아닌 값만 선택)
            tts_pitch_samples = pitch_values_tts[(time_steps_tts >= tts_word['start']) & (time_steps_tts <= tts_word['end'])]
            tts_time_samples = time_steps_tts[(time_steps_tts >= tts_word['start']) & (time_steps_tts <= tts_word['end'])]
            tts_pitch_samples = tts_pitch_samples[tts_pitch_samples > 0]  # 0이 아닌 값만 포함

            if len(tts_pitch_samples) > 0:
                tts_pitch_max = np.max(tts_pitch_samples)
                tts_pitch_min = np.min(tts_pitch_samples)
                tts_pitch_diff = tts_pitch_max - tts_pitch_min
                tts_max_time = tts_time_samples[np.argmax(tts_pitch_samples)]  # 최대 피치에 해당하는 시간

                # 피치 차이가 임계값 초과하는 경우 결과 저장
                if user_pitch_diff > threshold or tts_pitch_diff > threshold:
                    results.append({
                        'user_word': user_word['word'],
                        'tts_word': tts_word['word'],
                        'user_diff': user_pitch_diff,
                        'user_max_pitch': user_pitch_max,
                        'user_max_time': user_max_time,
                        'user_min_pitch': user_pitch_min,
                        'tts_diff': tts_pitch_diff,
                        'tts_max_pitch': tts_pitch_max,
                        'tts_max_time': tts_max_time,
                        'tts_min_pitch': tts_pitch_min
                    })

    return results



In [20]:

# 사용 예시
results = calculate_pitch_differences(
    word_intervals, tts_word_intervals, pitch_values, time_steps, pitch_values_tts, time_steps_tts
)

# 결과 출력
for result in results:
    print(f"사용자 단어: {result['user_word']}, TTS 단어: {result['tts_word']}, "
          f"사용자 피치 차이: {result['user_diff']}, "
          f"사용자 최대 피치: {result['user_max_pitch']} (시간: {result['user_max_time']}s), "
          f"사용자 최소 피치: {result['user_min_pitch']}, "
          f"TTS 피치 차이: {result['tts_diff']}, "
          f"TTS 최대 피치: {result['tts_max_pitch']} (시간: {result['tts_max_time']}s), "
          f"TTS 최소 피치: {result['tts_min_pitch']}")


사용자 단어: 안녕하세요., TTS 단어: 안녕하세요., 사용자 피치 차이: 26.652282432947175, 사용자 최대 피치: 121.71412670624733 (시간: 0.4037708333333334s), 사용자 최소 피치: 95.06184427330015, TTS 피치 차이: 61.73858293942588, TTS 최대 피치: 167.22702535645908 (시간: 0.32499999999999957s), TTS 최소 피치: 105.4884424170332
사용자 단어: 지금, TTS 단어: 지금, 사용자 피치 차이: 26.707513572415223, 사용자 최대 피치: 105.0815218400048 (시간: 1.6537708333333334s), 사용자 최소 피치: 78.37400826758957, TTS 피치 차이: 32.392062147681386, TTS 최대 피치: 137.3084031376915 (시간: 1.4849999999999994s), TTS 최소 피치: 104.91634099001011
사용자 단어: 서울말을, TTS 단어: 서울말을, 사용자 피치 차이: 19.49460050540378, 사용자 최대 피치: 117.27997824844142 (시간: 1.9337708333333334s), 사용자 최소 피치: 97.78537774303764, TTS 피치 차이: 53.053292289632594, TTS 최대 피치: 182.2511313371797 (시간: 2.0449999999999995s), TTS 최소 피치: 129.1978390475471
사용자 단어: 구사하는, TTS 단어: 구사하는, 사용자 피치 차이: 13.344675984213808, 사용자 최대 피치: 99.47655453382714 (시간: 2.4137708333333334s), 사용자 최소 피치: 86.13187854961333, TTS 피치 차이: 40.33673699350402, TTS 최대 피치: 142.71155223479207 (시간: 2.54

3. 기울기 평균으로 구하기

In [21]:
import numpy as np

# Gradient 계산 함수
def calculate_pitch_gradients(pitch_values, time_steps):
    gradients = []

    # 피치 기울기 계산
    for i in range(1, len(pitch_values)):
        if pitch_values[i] > 0 and pitch_values[i-1] > 0:  # 피치가 0보다 큰 경우에만 계산
            gradient = (pitch_values[i] - pitch_values[i-1]) / (time_steps[i] - time_steps[i-1])
            gradients.append(gradient)

    return gradients

# 세그먼트 기울기 비교 함수
def compare_segments(user_segments, tts_segments, user_pitch_values, user_time_steps, tts_pitch_values, tts_time_steps):
    highest_segment = None
    lowest_segment = None
    highest_difference = -float('inf')  # Initialize to negative infinity for maximum comparison
    lowest_difference = float('inf')    # Initialize to positive infinity for minimum comparison

    for user_segment, tts_segment in zip(user_segments, tts_segments):
        # 해당 세그먼트의 피치 값 및 타임스탬프 추출
        user_indices = [i for i, t in enumerate(user_time_steps) if user_segment['start'] <= t <= user_segment['end']]
        tts_indices = [i for i, t in enumerate(tts_time_steps) if tts_segment['start'] <= t <= tts_segment['end']]

        # 피치 기울기 계산
        if user_indices and tts_indices:
            user_segment_pitch = user_pitch_values[user_indices[0]:user_indices[-1] + 1]
            user_segment_time = user_time_steps[user_indices[0]:user_indices[-1] + 1]
            tts_segment_pitch = tts_pitch_values[tts_indices[0]:tts_indices[-1] + 1]
            tts_segment_time = tts_time_steps[tts_indices[0]:tts_indices[-1] + 1]

            user_gradients = calculate_pitch_gradients(user_segment_pitch, user_segment_time)
            tts_gradients = calculate_pitch_gradients(tts_segment_pitch, tts_segment_time)

            # 기울기 차이 계산
            if user_gradients and tts_gradients:
                avg_user_gradient = np.mean(user_gradients)
                avg_tts_gradient = np.mean(tts_gradients)
                difference = avg_user_gradient - avg_tts_gradient

                # 가장 높은 기울기 차이를 기록 (양의 방향)
                if difference > highest_difference:
                    highest_difference = difference
                    highest_segment = (user_segment, tts_segment, avg_user_gradient, avg_tts_gradient)

                # 가장 낮은 기울기 차이를 기록 (음의 방향)
                if difference < lowest_difference:
                    lowest_difference = difference
                    lowest_segment = (user_segment, tts_segment, avg_user_gradient, avg_tts_gradient)

    return highest_segment, lowest_segment




In [25]:

# 세그먼트 비교
highest_segment, lowest_segment = compare_segments(
    word_intervals, tts_word_intervals, pitch_values, time_steps, pitch_values_tts, time_steps_tts
)

# 결과 출력
if highest_segment:
    user_segment_high, tts_segment_high, avg_user_gradient_high, avg_tts_gradient_high = highest_segment
    print(f"음성이 TTS 대비 가장 높은 기울기를 가진 세그먼트:")
    print(f"사용자 세그먼트: {user_segment_high}, TTS 세그먼트: {tts_segment_high}")
    print(f"사용자 평균 기울기: {avg_user_gradient_high:.4f}, TTS 평균 기울기: {avg_tts_gradient_high:.4f}")
    print("음성이 TTS 대비 기울기가 높아요.")

if lowest_segment:
    user_segment_low, tts_segment_low, avg_user_gradient_low, avg_tts_gradient_low = lowest_segment
    print(f"음성이 TTS 대비 가장 낮은 기울기를 가진 세그먼트:")
    print(f"사용자 세그먼트: {user_segment_low}, TTS 세그먼트: {tts_segment_low}")
    print(f"사용자 평균 기울기: {avg_user_gradient_low:.4f}, TTS 평균 기울기: {avg_tts_gradient_low:.4f}")
    print("음성이 TTS 대비 기울기가 낮아요.")
else:
    print("세그먼트 비교에서 차이를 찾을 수 없습니다.")

음성이 TTS 대비 가장 높은 기울기를 가진 세그먼트:
사용자 세그먼트: {'word': '어디까지', 'start': 4.253, 'end': 4.654}, TTS 세그먼트: {'word': '어디까지', 'start': 4.369, 'end': 4.911}
사용자 평균 기울기: -80.0765, TTS 평균 기울기: -179.3498
음성이 TTS 대비 기울기가 높아요.
음성이 TTS 대비 가장 낮은 기울기를 가진 세그먼트:
사용자 세그먼트: {'word': '지금', 'start': 1.565, 'end': 1.765}, TTS 세그먼트: {'word': '지금', 'start': 1.476, 'end': 1.737}
사용자 평균 기울기: -241.9493, TTS 평균 기울기: -22.2522
음성이 TTS 대비 기울기가 낮아요.


4. DTW로 유사도 점수

In [28]:
!pip install fastdtw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp310-cp310-linux_x86_64.whl size=512555 sha256=3e4a33b73b69838999d2e6af5c9bfd5611e99e8bd0724cd35429db6293a682dd
  Stored in directory: /root/.cache/pip/wheels/73/c8/f7/c25448dab74c3acf4848bc25d513c736bb93910277e1528ef4
Successfully built fastdtw


In [29]:
import numpy as np
import librosa
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

def extract_features(audio_file):
    # Load audio file
    y, sr = librosa.load(audio_file, sr=None)

    # Extract MFCCs
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

    # Extract pitch (fundamental frequency)
    pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
    pitch_values = pitches[pitches > 0]
    if len(pitch_values) > 0:
        pitch_mean = np.mean(pitch_values)  # Average pitch value
    else:
        pitch_mean = 0  # Handle cases with no pitch

    return mfccs, pitch_mean

def calculate_similarity(mfcc1, pitch1, mfcc2, pitch2):
    # Calculate DTW distance between MFCCs
    distance, _ = fastdtw(mfcc1.T, mfcc2.T, dist=euclidean)

    # Normalize DTW distance to a similarity score
    max_distance = max(mfcc1.shape[1], mfcc2.shape[1]) * np.max([np.max(np.abs(mfcc1)), np.max(np.abs(mfcc2))])
    dtw_similarity = max(0, 1 - (distance / (max_distance + 1e-9)))  # Add small constant to avoid division by zero

    # Calculate pitch difference as a similarity
    pitch_diff = abs(pitch1 - pitch2)
    max_pitch_range = 400  # Adjust this range if necessary for normalization
    normalized_pitch_diff = max(0, 1 - (pitch_diff / max_pitch_range))

    # Combine similarities with weighted averaging
    similarity_score = 0.7 * dtw_similarity + 0.3 * normalized_pitch_diff

    # Scale similarity score to a 0-100 range
    return similarity_score * 100

def evaluate_score(score):
    if score >= 90:
        return "Excellent! You're singing almost perfectly."
    elif score >= 75:
        return "Good job! You're very close."
    elif score >= 50:
        return "Not bad! Keep practicing."
    elif score >= 30:
        return "Needs improvement. Try again!"
    else:
        return "Keep practicing! Don't give up!"

def main(user_audio_file, tts_audio_file):
    # Extract features from both audio files
    mfcc_user, pitch_user = extract_features(user_audio_file)
    mfcc_tts, pitch_tts = extract_features(tts_audio_file)

    # Calculate similarity score
    similarity_score = calculate_similarity(mfcc_user, pitch_user, mfcc_tts, pitch_tts)

    # Ensure the score is capped between 0 and 100
    similarity_score = min(max(similarity_score, 0), 100)

    # Evaluate the score
    evaluation = evaluate_score(similarity_score)

    print(f"Similarity Score: {similarity_score:.2f}/100")
    print(evaluation)



In [30]:

# 예시 사용법
user_audio_file = 'real_test_only.wav'  # 사용자 음성 파일
tts_audio_file = 'seoul_test4.wav'  # TTS 음성 파일

main(user_audio_file, tts_audio_file)


Similarity Score: 56.08/100
Not bad! Keep practicing.
